In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.executable

'/cluster/home/skyriakos/pyg_env/bin/python'

In [3]:
!which python

/cluster/home/skyriakos/pyg_env/bin/python


In [4]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import ogb
from tqdm import tqdm
import hiplot as hip
from copy import deepcopy
import datetime

In [5]:
import torch

print(f"{torch.__version__=}")
print(f"{torch.__file__=}")
print(f"{torch.cuda.device_count()=}")
print(f"{torch.cuda.is_available()=}")
print(f"{torch.version.cuda=}")

torch.__version__='1.11.0+cu115'
torch.__file__='/usr/local/lib/python3.8/dist-packages/torch/__init__.py'
torch.cuda.device_count()=5
torch.cuda.is_available()=True
torch.version.cuda='11.5'


In [6]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset, TensorDataset

In [7]:
cwd = os.getcwd()
print(cwd)
cwd_parent = os.path.abspath(os.path.join(cwd, os.pardir))
print(cwd_parent)

sys.path.append(cwd_parent)

/cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/notebooks
/cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git


In [8]:
import deepadr
from deepadr.dataset import *
from deepadr.utilities import *
from deepadr.run_workflow import *
from deepadr.chemfeatures import *
from deepadr.hyphelperv2 import *
from deepadr.model_gnn_ogb import GNN, DeepAdr_SiameseTrf, ExpressionNN
# from ogb.graphproppred import Evaluator

In [9]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [10]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'

In [11]:
report_available_cuda_devices()

number of GPUs available: 5
cuda:0, name:NVIDIA GeForce GTX 1080 Ti
total memory available: 10.91656494140625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:1, name:NVIDIA GeForce GTX 1080 Ti
total memory available: 10.91656494140625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:2, name:NVIDIA GeForce GTX 1080 Ti
total memory available: 10.91656494140625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:3, name:NVIDIA GeForce GTX 1080 Ti
total memory available: 10.91656494140625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:4, 

In [12]:
n_gpu = torch.cuda.device_count()
# n_gpu = 1
n_gpu

5

In [13]:
device_cpu = get_device(to_gpu=False)
# device_gpu = get_device(True, index=0)

# fdtype = torch.float32

In [14]:
print("torch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print(sys.version)

torch: 1.11.0+cu115
CUDA: 11.5
3.8.10 (default, Jun 22 2022, 20:18:18) 
[GCC 9.4.0]


### Preparing dataset 

In [15]:
# options: 
# 'total_thresh' + 4,3,2
# 'loewe_thresh', 'hsa_thresh', 'bliss_thresh', 'zip_thresh' + 1

score = 'total_thresh'
score_val = 4

In [16]:
DSdataset_name = f'DrugComb_{score}_{score_val}'

data_fname = 'data_v1' # v2 for baseline models

In [17]:
targetdata_dir = create_directory(os.path.join(processed_dir, DSdataset_name, data_fname))
targetdata_dir_raw = create_directory(os.path.join(targetdata_dir, "raw"))
targetdata_dir_processed = create_directory(os.path.join(targetdata_dir, "processed"))
targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
# # ReaderWriter.dump_data(dpartitions, os.path.join(targetdata_dir, 'data_partitions.pkl'))
print(targetdata_dir)

path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
/cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/data/processed/DrugComb_total_thresh_4/data_v1


In [18]:
%%time

# Make sure to first run the "DDoS_Dataset_Generation" notebook first

dataset = MoleculeDataset(root=targetdata_dir)

CPU times: user 7.28 ms, sys: 214 ms, total: 221 ms
Wall time: 234 ms


In [19]:
# print()
# print(f'Dataset: {dataset}:')
# print('====================')
# print(f'Number of graphs: {len(dataset)}')
# print(f'Number of features: {dataset.num_features}')
# print(f'Number of classes: {dataset.num_classes}')

In [20]:
# data0 = dataset[0]  # Get the first graph object.
# data0

In [21]:
used_dataset = dataset

# If you want to use a smaller subset of the dataset for testing
# smaller_dataset_len = int(len(dataset)/1)
# used_dataset = dataset[:smaller_dataset_len]

In [22]:
type(used_dataset)

deepadr.dataset.MoleculeDataset

In [23]:
dataset.data.y

tensor([0, 0, 0,  ..., 0, 1, 1], dtype=torch.int32)

In [24]:
fold_partitions = get_stratified_partitions(dataset.data.y,
                                            num_folds=5,
                                            valid_set_portion=0.1,
                                            random_state=42)

fold_num: 0
train data
class: 0 norm count: 0.5764128559102675
class: 1 norm count: 0.4235871440897325
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5764751552795031
class: 1 norm count: 0.4235248447204969

-------------------------
fold_num: 1
train data
class: 0 norm count: 0.5764128559102675
class: 1 norm count: 0.4235871440897325
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5764751552795031
class: 1 norm count: 0.4235248447204969

-------------------------
fold_num: 2
train data
class: 0 norm count: 0.5764356969533567
class: 1 norm count: 0.4235643030466433
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5763929334109882
class: 1 norm count: 0.4236070665890118

-------------------------
fold_num: 3
train data
class: 0 norm count: 0.576

In [25]:
print("Number of training graphs: "+ str(len(fold_partitions[0]['train'])))
print("Number of validation graphs: "+ str(len(fold_partitions[0]['validation'])))
print("Number of testing graphs: "+ str(len(fold_partitions[0]['test'])))

Number of training graphs: 18544
Number of validation graphs: 2061
Number of testing graphs: 5152


In [26]:
# print(dataset.num_classes)

In [27]:
# training params total_thresh 4
tp = {
    "batch_size" : 300,
    "num_epochs" : 5,
    
    "emb_dim" : 100,
    "gnn_type" : "gatv2",
    "num_layer" : 5,
    "graph_pooling" : "mean", #attention
    
    "input_embed_dim" : None,
    "gene_embed_dim": 1,
    "num_attn_heads" : 2,
    "num_transformer_units" : 1,
    "p_dropout" : 0.3,
#     "nonlin_func" : nn.ReLU(),
    "mlp_embed_factor" : 2,
    "pooling_mode" : 'attn',
    "dist_opt" : 'cosine',

    "base_lr" : 3e-4, #3e-4
    "max_lr_mul": 10,
    "l2_reg" : 1e-7,
    "loss_w" : 1.,
    "margin_v" : 1.,

    "expression_dim" : 64,
    "expression_input_size" : 908,
    "exp_H1" : 4096,
    "exp_H2" : 1024
}

In [28]:
# tp = {
#     "batch_size" : 32,
#     "num_epochs" : 5,
    
#     "emb_dim" : 100,
#     "gnn_type" : "gatv2",
#     "num_layer" : 5,
#     "graph_pooling" : "mean", #attention
    
#     "input_embed_dim" : None,
#     "gene_embed_dim": 20,
#     "num_attn_heads" : 2,
#     "num_transformer_units" : 1,
#     "p_dropout" : 0.3,
# #     "nonlin_func" : nn.ReLU(),
#     "mlp_embed_factor" : 2,
#     "pooling_mode" : 'attn',
#     "dist_opt" : 'cosine',

#     "base_lr" : 3e-4, #3e-4
#     "max_lr_mul": 10,
#     "l2_reg" : 1e-7,
#     "loss_w" : 1.,
#     "margin_v" : 1.,

#     "expression_dim" : 64,
#     "expression_input_size" : 908,
#     "exp_H1" : 4096,
#     "exp_H2" : 1024
# }

In [29]:
# # training params total_thresh 3
# tp = {
#     "batch_size" : 300,
#     "num_epochs" : 200,
    
#     "emb_dim" : 300,
#     "gnn_type" : "gatv2",
#     "num_layer" : 5,
#     "graph_pooling" : "mean", #attention
    
#     "input_embed_dim" : None,
#     "gene_embed_dim": 1,
#     "num_attn_heads" : 2,
#     "num_transformer_units" : 1,
#     "p_dropout" : 0.3,
# #     "nonlin_func" : nn.ReLU(),
#     "mlp_embed_factor" : 2,
#     "pooling_mode" : 'attn',
#     "dist_opt" : 'cosine',

#     "base_lr" : 3e-5, #3e-4
#     "max_lr_mul": 10,
#     "l2_reg" : 1e-8,
#     "loss_w" : 0.3,
#     "margin_v" : 1.,

#     "expression_dim" : 64,
#     "expression_input_size" : 908,
#     "exp_H1" : 500,
#     "exp_H2" : 400
# }

In [30]:
def spawn_q_process(q_process):
    print(">>> spawning hyperparam search process")
    q_process.start()
    
def join_q_process(q_process):
    q_process.join()
    print("<<< joined hyperparam search process")
    
def create_q_process(queue, used_dataset, gpu_num, tphp, exp_dir, partition):
#     fold_gpu_map = {0:gpu_num}
    return mp.Process(target=deepadr.hyphelperv2.run_exp, args=(queue, used_dataset, gpu_num, tphp, exp_dir, partition))

In [31]:
n_gpu

5

In [32]:
range(min(n_gpu, len(fold_partitions)))

range(0, 5)

In [33]:
import torch.multiprocessing as mp
mp.set_start_method("spawn", force=True)

queue = mp.Queue()
q_processes = []

# partition = fold_partitions[0]
time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

print("Start: " + time_stamp)

for q_i in range(min(n_gpu, len(fold_partitions))):
#     device_gpu = get_device(True, index=q_i)
    partition = fold_partitions[q_i]
    exp_dir = create_directory(os.path.join(targetdata_dir_exp, "fold_"+str(q_i)+"_"+time_stamp))
    create_directory(os.path.join(exp_dir, "predictions"))
    create_directory(os.path.join(exp_dir, "modelstates"))


#     tphp = generate_tp_hp(tp, hyperparam_space[q_i], hp_names)
    
    q_process = create_q_process(queue, dataset, q_i, tp, exp_dir, partition)
    q_processes.append(q_process)
    spawn_q_process(q_process)

spawned_processes = n_gpu
    
# for q_i in range(len(hyperparam_space)):
for q_i in range(min(n_gpu, len(fold_partitions))):
    join_q_process(q_processes[q_i])
    released_gpu_num = queue.get()
    print("released_gpu_num:", released_gpu_num)
#     if(spawned_processes < len(hyperparam_space)):
# #         device_gpu = get_device(True, index=q_i)
#         time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
#         exp_dir = create_directory(os.path.join(targetdata_dir_exp, "exp_"+str(q_i)+"_"+time_stamp))
#         tphp = generate_tp_hp(tp, hyperparam_space[q_i], hp_names)

#         q_process = create_q_process(queue, used_dataset, released_gpu_num, tphp, exp_dir, partition)
#         q_processes.append(q_process)
#         spawn_q_process(q_process)
#         spawned_processes = spawned_processes + 1

Start: 2022-09-05_12-25-54
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
>>> spawning hyperparam search process
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
>>> spawning hyperparam search process
gpu: cuda:0
Dropout(p=0.3, inplace=False) Dropout(p=0.3, inplace=False)
=====Epoch 0
Training...


Iteration:   0%|          | 0/62 [00:00<?, ?it/s]

path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
>>> spawning hyperparam search process


Iteration:   5%|▍         | 3/62 [00:04<01:05,  1.11s/it]

gpu: cuda:1
Dropout(p=0.3, inplace=False) Dropout(p=0.3, inplace=False)
=====Epoch 0
Training...


...

{'Train':  best_epoch_indx:4
 auc:0.8521875861543325 
 apur:0.8182335767885369 
 f1:0.7386709654307719 
 precision:0.6955250730829773 
 recall:0.7875238701464036 
, 'Validation':  best_epoch_indx:4
 auc:0.8450233530416806 
 apur:0.8079170324114939 
 f1:0.7244179750947483 
 precision:0.686858316221766 
 recall:0.7663230240549829 
}
Finished training and validating!


Iteration:  44%|████▍     | 8/18 [00:00<00:00, 11.08it/s]

<<< joined hyperparam search process
released_gpu_num: 3


Iteration: 100%|██████████| 18/18 [00:01<00:00, 13.39it/s]


{'Test':  best_epoch_indx:4
 auc:0.8420648411217783 
 apur:0.8096406186468994 
 f1:0.7232472324723247 
 precision:0.6870103092783505 
 recall:0.763519706691109 
}
Finished testing!
<<< joined hyperparam search process
released_gpu_num: 4


In [34]:
print("End: " + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))


End: 2022-09-05_12-27-15
